In [14]:
# Importar las bibliotecas necesarias
from ultralytics import YOLO
import pydicom
import numpy as np
from PIL import Image, ImageDraw

# Ruta al modelo entrenado
model_path = '/Users/macdenico/PycharmProjects/PMM-FALP2024/Model Training and Validation/deteccion/modelos/best.pt'

# Ruta a la imagen DICOM
image_path = '/Users/macdenico/PycharmProjects/PMM-FALP2024/Model Training and Validation/deteccion/images/2fc787f72bfa6b6c0bbc0a85413c8a9f.dicom'

# Cargar el modelo YOLO
model = YOLO(model_path)

# Leer la imagen DICOM
dicom = pydicom.dcmread(image_path)

# Obtener el arreglo de píxeles de la imagen DICOM
pixel_array = dicom.pixel_array

# Normalizar el arreglo de píxeles a valores entre 0 y 255
pixel_array = pixel_array.astype(float)
scaled_pixel_array = (np.maximum(pixel_array - np.min(pixel_array), 0) / (np.max(pixel_array) - np.min(pixel_array))) * 255.0
scaled_pixel_array = np.uint8(scaled_pixel_array)

# Manejar la interpretación fotométrica MONOCHROME1 si es necesario
if dicom.PhotometricInterpretation == "MONOCHROME1":
    scaled_pixel_array = np.max(scaled_pixel_array) - scaled_pixel_array

# Convertir el arreglo de píxeles en una imagen PIL
image = Image.fromarray(scaled_pixel_array)

# Si la imagen es en escala de grises, convertirla a RGB
if image.mode != 'RGB':
    image = image.convert('RGB')

# Realizar la inferencia utilizando el modelo YOLO
result = model.predict(image, conf=0.2, verbose=False, imgsz=1280)[0]

# Mostrar los resultados de la inferencia
print(result)

# Obtener las cajas delimitadoras de las detecciones
boxes = result.boxes

# Crear una máscara vacía con las mismas dimensiones que la imagen original
mask = Image.new('L', image.size, 0)  # 'L' para modo de imagen en escala de grises

# Dibujar las cajas delimitadoras en la máscara
if boxes is not None and len(boxes) > 0:
    # Convertir las cajas a un array de NumPy
    boxes_array = boxes.xyxy.cpu().numpy()  # Convertir a CPU y luego a NumPy

    draw = ImageDraw.Draw(mask)

    for box in boxes_array:
        # Obtener las coordenadas de la caja y asegurarse de que son enteros
        x1, y1, x2, y2 = map(int, box[:4])
        # Dibujar un rectángulo lleno (valor 255) en la máscara
        draw.rectangle([x1, y1, x2, y2], outline=255, fill=255)

    # Superponer la máscara coloreada sobre la imagen original
    # Convertir la imagen original a modo RGBA
    image_rgba = image.convert('RGBA')

    # Crear una imagen del color de la máscara (por ejemplo, rojo semitransparente)
    mask_color = (255, 0, 0, 100)  # (R, G, B, Alpha)

    # Crear una imagen del color de la máscara con transparencia
    colored_mask = Image.new('RGBA', image.size, mask_color)

    # Usar la máscara como canal alfa
    mask_np = np.array(mask)
    alpha_mask = (mask_np / 255) * mask_color[3]  # Escalar el valor de alfa según la máscara

    # Reemplazar el canal alfa en la máscara coloreada
    colored_mask.putalpha(Image.fromarray(alpha_mask.astype('uint8')))

    # Superponer la máscara coloreada sobre la imagen original
    combined = Image.alpha_composite(image_rgba, colored_mask)

    # Mostrar la imagen combinada
    combined.show()

    # Guardar la imagen combinada si lo deseas
    #combined.save('/ruta/donde/guardar/imagen_con_mascara.png')

else:
    print("No se encontraron detecciones en la imagen.")


ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'mass'}
obb: None
orig_img: array([[[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [62, 62, 62],
        [63, 63, 63],
        [64, 64, 64]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [61, 61, 61],
        [62, 62, 62],
        [62, 62, 62]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [62, 62, 62],
        [63, 63, 63],
        [64, 64, 64]],

       ...,

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,

In [17]:
# Importar las bibliotecas necesarias
from ultralytics import YOLO
import pydicom
import numpy as np
from PIL import Image, ImageDraw
from pydicom.pixel_data_handlers.util import apply_voi_lut
import logging

# Configurar el logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Ruta al modelo entrenado
model_path = '/Users/macdenico/PycharmProjects/PMM-FALP2024/Model Training and Validation/deteccion/modelos/best.pt'

# Ruta a la imagen DICOM
image_path = '/Users/macdenico/PycharmProjects/PMM-FALP2024/Model Training and Validation/deteccion/images/2fc787f72bfa6b6c0bbc0a85413c8a9f.dicom'

# Cargar el modelo YOLO
logger.info("Cargando el modelo YOLO...")
model = YOLO(model_path)
logger.info("Modelo YOLO cargado exitosamente.")

def procesar_imagen_dicom(image_path):
    """
    Procesa una imagen DICOM y la convierte a una imagen PIL en escala de grises ('L').
    Aplica VOI LUT, maneja la interpretación fotométrica y normaliza la imagen.
    """
    try:
        # Leer el dataset DICOM
        logger.info(f"Leyendo el archivo DICOM: {image_path}")
        dicom = pydicom.dcmread(image_path)
        
        # Aplicar VOI LUT para ajustar el contraste y brillo
        logger.info("Aplicando VOI LUT...")
        image_data = apply_voi_lut(dicom.pixel_array, dicom)
        
        # Manejar la interpretación fotométrica
        photometric = dicom.get('PhotometricInterpretation', 'MONOCHROME2')
        logger.info(f"Interpretación Fotométrica: {photometric}")
        if photometric == "MONOCHROME1":
            logger.info("Invertiendo la imagen para convertir de MONOCHROME1 a MONOCHROME2.")
            image_data = np.max(image_data) - image_data
            photometric = "MONOCHROME2"
        
        # Normalizar la imagen a un rango de 0 a 1
        logger.info("Normalizando la imagen...")
        image_normalized = image_data - np.min(image_data)
        if np.max(image_normalized) != 0:
            image_normalized = image_normalized / np.max(image_normalized)
        else:
            logger.warning("La imagen tiene un rango de intensidad nulo. Se establecerá a cero.")
            image_normalized = np.zeros(image_normalized.shape)
        
        # Convertir a uint8 (0-255)
        logger.info("Escalando la imagen a 0-255...")
        image_uint8 = (image_normalized * 255).astype(np.uint8)
        
        # Convertir a imagen PIL en modo 'L' (escala de grises)
        logger.info("Convirtiendo a imagen PIL en modo 'L' (escala de grises).")
        image_pil = Image.fromarray(image_uint8).convert('L')
        
        return image_pil
    
    except Exception as e:
        logger.error(f"Error al procesar la imagen DICOM: {e}")
        return None

def realizar_inferencia_y_dibujar_detecciones(image_pil, model):
    """
    Realiza la inferencia utilizando el modelo YOLO y dibuja las detecciones en la imagen.
    """
    try:
        # Convertir la imagen PIL en RGB para YOLO
        logger.info("Convirtiendo la imagen a RGB para la inferencia con YOLO.")
        image_rgb = image_pil.convert('RGB')
        
        # Realizar la inferencia utilizando el modelo YOLO
        logger.info("Realizando la inferencia con el modelo YOLO...")
        result = model.predict(image_rgb, conf=0.2, verbose=False, imgsz=1280)[0]
        logger.info("Inferencia completada.")
        
        # Mostrar los resultados de la inferencia
        logger.info(f"Resultados de la inferencia: {result}")
        
        # Obtener las cajas delimitadoras de las detecciones
        boxes = result.boxes
        
        # Crear una máscara vacía con las mismas dimensiones que la imagen original
        logger.info("Creando la máscara de detecciones.")
        mask = Image.new('L', image_pil.size, 0)  # 'L' para modo de imagen en escala de grises
        
        # Dibujar las cajas delimitadoras en la máscara
        if boxes is not None and len(boxes) > 0:
            logger.info(f"Se encontraron {len(boxes)} detecciones.")
            # Convertir las cajas a un array de NumPy
            boxes_array = boxes.xyxy.cpu().numpy()  # Convertir a CPU y luego a NumPy
            
            draw = ImageDraw.Draw(mask)
            
            for box in boxes_array:
                # Obtener las coordenadas de la caja y asegurarse de que son enteros
                x1, y1, x2, y2 = map(int, box[:4])
                # Dibujar un rectángulo lleno (valor 255) en la máscara
                draw.rectangle([x1, y1, x2, y2], outline=255, fill=255)
            
            # Superponer la máscara coloreada sobre la imagen original
            logger.info("Superponiendo la máscara de detecciones sobre la imagen original.")
            # Convertir la imagen original a modo RGBA
            image_rgba = image_rgb.convert('RGBA')
            
            # Crear una imagen del color de la máscara (por ejemplo, rojo semitransparente)
            mask_color = (255, 0, 0, 100)  # (R, G, B, Alpha)
            
            # Crear una imagen del color de la máscara con transparencia
            colored_mask = Image.new('RGBA', image_pil.size, mask_color)
            
            # Usar la máscara como canal alfa
            mask_np = np.array(mask)
            alpha_mask = (mask_np / 255) * mask_color[3]  # Escalar el valor de alfa según la máscara
            
            # Reemplazar el canal alfa en la máscara coloreada
            colored_mask.putalpha(Image.fromarray(alpha_mask.astype('uint8')))
            
            # Superponer la máscara coloreada sobre la imagen original
            combined = Image.alpha_composite(image_rgba, colored_mask)
            
            # Mostrar la imagen combinada
            logger.info("Mostrando la imagen con detecciones superpuestas.")
            combined.show()
            
            # Guardar la imagen combinada si lo deseas
            # combined.save('/ruta/donde/guardar/imagen_con_mascara.png')
        
        else:
            logger.info("No se encontraron detecciones en la imagen.")
            image_rgb.show()
    
    except Exception as e:
        logger.error(f"Error durante la inferencia o el procesamiento de detecciones: {e}")

def main():
    # Procesar la imagen DICOM
    image_pil = procesar_imagen_dicom(image_path)
    
    if image_pil is not None:
        # Realizar la inferencia y dibujar las detecciones
        realizar_inferencia_y_dibujar_detecciones(image_pil, model)
    else:
        logger.error("No se pudo procesar la imagen DICOM. Terminando la ejecución.")

if __name__ == "__main__":
    main()


INFO:__main__:Cargando el modelo YOLO...
INFO:__main__:Modelo YOLO cargado exitosamente.
INFO:__main__:Leyendo el archivo DICOM: /Users/macdenico/PycharmProjects/PMM-FALP2024/Model Training and Validation/deteccion/images/2fc787f72bfa6b6c0bbc0a85413c8a9f.dicom
INFO:__main__:Aplicando VOI LUT...
INFO:__main__:Interpretación Fotométrica: MONOCHROME2
INFO:__main__:Normalizando la imagen...
INFO:__main__:Escalando la imagen a 0-255...
INFO:__main__:Convirtiendo a imagen PIL en modo 'L' (escala de grises).
INFO:__main__:Convirtiendo la imagen a RGB para la inferencia con YOLO.
INFO:__main__:Realizando la inferencia con el modelo YOLO...
INFO:__main__:Inferencia completada.
INFO:__main__:Resultados de la inferencia: ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'mass'}
obb: None
orig_img: array([[[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 9,  9,  9],
     